### Ana Carolina Junger
Atividade 2 INF2137

## 1. Definição do Problema

O Dataset utilizado neste projeto é o Fake.Br Corpus, o primeiro corpus de fake news em português [Monteiro et al., 2018]. As informações foram extraídas do seu projeto no github (https://github.com/roneysco/Fake.br-Corpus). Nele, temos 3 versões do corpus: a primeira com os textos por completo + seus metadados, a segunda com os textos já pre processados pelos autores e a terceira com os pares de fake e true news truncados para terem o mesmo tamanho. Nessa segunda parte do projeto, iremos avaliar os dados pré processados.

# 2. Iniciando o notebook + imports

In [2]:

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as ms # para tratamento de missings
from matplotlib import cm
from pandas import set_option
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import warnings
warnings.filterwarnings("ignore")

# 3. Carregando o Dataset

Esse arquivo já está pre-processado, normalizando as palavras, removendo stopwords, acentos etc

In [3]:
url = "https://raw.githubusercontent.com/roneysco/Fake.br-Corpus/master/preprocessed/pre-processed.csv"

df_orig = pd.read_csv(url, skiprows=0, delimiter=',', usecols=["label", "preprocessed_news"])

In [4]:
df = df_orig.copy()

In [18]:
vectorizer = CountVectorizer()
X_processed = vectorizer.fit_transform(df["preprocessed_news"])

tfidfconverter = TfidfVectorizer(max_features=800000)
X_processed_Tfid = tfidfconverter.fit_transform(df["preprocessed_news"])

len(vectorizer.get_feature_names_out())

79541

## 6. Modelos de Classificação

Já é sabido pelo modelo que o SVM performa muito bem com o bag-of-words, dessa forma iremos avaliar o variações do bag-of-words em diferentes modelos. De maneira analoga, também iremos avaliar o TFIDF.

#### Definição da classe do Pre Processador

In [45]:
# definindo o corte e a semente 
test_size = 0.30
seed = 7

X_train, X_test, y_train, y_test = train_test_split(X_processed.toarray(), df["label"], test_size=test_size, random_state=seed)


model = GaussianNB()
model.fit(X_train, y_train)

# # X_train_transform, X_test_transform, y_train_transform, y_test_transform = pre_processador.pre_processar(df, test_size)
# # Parâmetros e partições da validação cruzada
scoring = 'accuracy'
num_particoes = 10

kfold = StratifiedKFold(n_splits=num_particoes, shuffle=True, random_state=seed)

In [12]:
# definindo o corte e a semente 
test_size = 0.30
seed = 7

X_train, X_test, y_train, y_test = train_test_split(X_processed_Tfid.toarray(), df["label"], test_size=test_size, random_state=seed)


model = GaussianNB()
model.fit(X_train, y_train)

# # # X_train_transform, X_test_transform, y_train_transform, y_test_transform = pre_processador.pre_processar(df, test_size)
# # # Parâmetros e partições da validação cruzada
# scoring = 'accuracy'
# num_particoes = 10

# kfold = StratifiedKFold(n_splits=num_particoes, shuffle=True, random_state=seed)

GaussianNB()

In [13]:
predictions = model.predict(X_test)
print(accuracy_score(y_test, predictions))
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

0.8023148148148148
[[753 345]
 [ 82 980]]
              precision    recall  f1-score   support

        fake       0.90      0.69      0.78      1098
        true       0.74      0.92      0.82      1062

    accuracy                           0.80      2160
   macro avg       0.82      0.80      0.80      2160
weighted avg       0.82      0.80      0.80      2160



### Analisando os melhores parametros para cada modelo

In [19]:
# variações 

# definindo o corte e a semente 
test_size = 0.20
seed = 7
# Parâmetros e partições da validação cruzada
scoring = 'accuracy'
num_particoes = 10

kfold = StratifiedKFold(n_splits=num_particoes, shuffle=True, random_state=seed)

max_features = [1000, 5000, 10000] # max geral é 79541

In [20]:
# Processo de GridSearch 

models = []

# Navie Bayes 
param_grid = {'var_smoothing': np.logspace(0,-9, num=20)}
models.append(("Navie Bayes", GaussianNB(), param_grid))


# Decision Tree
criterion = ['gini','entropy']
max_depth = [3,4,5,6,7,8,9,10,11,12,15,20,30,40,50,70,90,120,150]
param_grid = {'criterion': criterion,'max_depth': max_depth}
models.append(("Decision Tree", DecisionTreeClassifier(), param_grid))


#  KNN
k = [1,3,5,7,9,11,13,15,17,19,21]
distancias = ["euclidean", "manhattan", "minkowski"]
param_grid = dict(n_neighbors=k, metric=distancias)
models.append(("KNN", KNeighborsClassifier(), param_grid))

# SVM
param_grid = {
  'C': [0.1, 0.3, 0.5, 0.7, 0.9, 1.0, 1.3, 1.5, 1.7, 2.0],
  'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
}
models.append(("SVM", SVC(), param_grid))

In [23]:
for max_feature in max_features:
  bag_of_words = CountVectorizer(max_features=max_feature)
  X_processed = bag_of_words.fit_transform(df["preprocessed_news"])
  
  X_train, X_test, y_train, y_test = train_test_split(X_processed.toarray(), df["label"], test_size=test_size, random_state=seed)
  print("BAG-OF-WORDS MAX-FEATURE: ", max_feature)
  
  for name, model, param_grid in models:
    grid = GridSearchCV(estimator=model, param_grid=param_grid, 
                      scoring=scoring, cv=kfold)
    grid.fit(X_train, y_train) 

    # imprime o melhor resultado
    print("Melhor %s : %f usando %s" % 
          (name, grid.best_score_, grid.best_params_))
    
    
  tfidfconverter = TfidfVectorizer(max_features=max_feature)
  X_processed_Tfid = tfidfconverter.fit_transform(df["preprocessed_news"])
  X_train, X_test, y_train, y_test = train_test_split(X_processed_Tfid.toarray(), df["label"], test_size=test_size, random_state=seed)
  print("TFIDF MAX-FEATURE: ", max_feature)
  
  
  for name, model, param_grid in models:
    grid = GridSearchCV(estimator=model, param_grid=param_grid, 
                      scoring=scoring, cv=kfold)
    grid.fit(X_train, y_train) 

    # imprime o melhor resultado
    print("Melhor %s : %f usando %s" % 
          (name, grid.best_score_, grid.best_params_))

BAG-OF-WORDS MAX-FEATURE:  1000
Melhor Navie Bayes : 0.947917 usando {'var_smoothing': 1.8329807108324375e-05}
Melhor Decision Tree : 0.907986 usando {'criterion': 'entropy', 'max_depth': 150}
Melhor KNN : 0.807639 usando {'metric': 'euclidean', 'n_neighbors': 1}
Melhor SVM : 0.961806 usando {'C': 2.0, 'kernel': 'rbf'}
TFIDF MAX-FEATURE:  1000
Melhor Navie Bayes : 0.839410 usando {'var_smoothing': 0.012742749857031341}
Melhor Decision Tree : 0.902778 usando {'criterion': 'gini', 'max_depth': 7}
Melhor KNN : 0.734375 usando {'metric': 'euclidean', 'n_neighbors': 1}
Melhor SVM : 0.958681 usando {'C': 2.0, 'kernel': 'rbf'}
BAG-OF-WORDS MAX-FEATURE:  5000
Melhor Navie Bayes : 0.942187 usando {'var_smoothing': 1.8329807108324375e-05}
Melhor Decision Tree : 0.910069 usando {'criterion': 'entropy', 'max_depth': 12}
Melhor KNN : 0.760938 usando {'metric': 'euclidean', 'n_neighbors': 1}
Melhor SVM : 0.962847 usando {'C': 1.7, 'kernel': 'rbf'}
TFIDF MAX-FEATURE:  5000
Melhor Navie Bayes : 0.8190

BAG-OF-WORDS MAX-FEATURE:  1000
Melhor Navie Bayes : 0.947917 usando {'var_smoothing': 1.8329807108324375e-05}
Melhor Decision Tree : 0.907986 usando {'criterion': 'entropy', 'max_depth': 150}
Melhor KNN : 0.807639 usando {'metric': 'euclidean', 'n_neighbors': 1}
Melhor SVM : 0.961806 usando {'C': 2.0, 'kernel': 'rbf'}
TFIDF MAX-FEATURE:  1000
Melhor Navie Bayes : 0.839410 usando {'var_smoothing': 0.012742749857031341}
Melhor Decision Tree : 0.902778 usando {'criterion': 'gini', 'max_depth': 7}
Melhor KNN : 0.734375 usando {'metric': 'euclidean', 'n_neighbors': 1}
Melhor SVM : 0.958681 usando {'C': 2.0, 'kernel': 'rbf'}
BAG-OF-WORDS MAX-FEATURE:  5000
Melhor Navie Bayes : 0.942187 usando {'var_smoothing': 1.8329807108324375e-05}
Melhor Decision Tree : 0.910069 usando {'criterion': 'entropy', 'max_depth': 12}
Melhor KNN : 0.760938 usando {'metric': 'euclidean', 'n_neighbors': 1}
Melhor SVM : 0.962847 usando {'C': 1.7, 'kernel': 'rbf'}
TFIDF MAX-FEATURE:  5000
Melhor Navie Bayes : 0.819097 usando {'var_smoothing': 0.0379269019073225}
Melhor Decision Tree : 0.901563 usando {'criterion': 'gini', 'max_depth': 7}
Melhor KNN : 0.726910 usando {'metric': 'euclidean', 'n_neighbors': 1}
Melhor SVM : 0.959896 usando {'C': 0.7, 'kernel': 'linear'}
BAG-OF-WORDS MAX-FEATURE:  10000
Melhor Navie Bayes : 0.935590 usando {'var_smoothing': 1.8329807108324375e-05}
Melhor Decision Tree : 0.909028 usando {'criterion': 'entropy', 'max_depth': 12}
Melhor KNN : 0.746354 usando {'metric': 'euclidean', 'n_neighbors': 1}
Melhor SVM : 0.962674 usando {'C': 2.0, 'kernel': 'rbf'}
TFIDF MAX-FEATURE:  10000
Melhor Navie Bayes : 0.821181 usando {'var_smoothing': 0.0379269019073225}
Melhor Decision Tree : 0.900347 usando {'criterion': 'entropy', 'max_depth': 11}
Melhor KNN : 0.723437 usando {'metric': 'euclidean', 'n_neighbors': 1}
Melhor SVM : 0.958160 usando {'C': 2.0, 'kernel': 'rbf'}
BAG-OF-WORDS MAX-FEATURE:  50000
Melhor Navie Bayes : 0.898958 usando {'var_smoothing': 6.158482110660267e-06}
Melhor Decision Tree : 0.910417 usando {'criterion': 'entropy', 'max_depth': 40}


O Algoritimo rodou até 10000 features depois travou o computador, vamos considerar esse o valor máximo possível de manipular


### Criação e avaliação dos modelos

In [21]:
np.random.seed(7) # definindo uma semente global

# armazeando os pipelines e os resultados para todas as visões do dataset
# como sao datasets diferentes, pelos tratamentos, irei definir dois pipelines 
pipelines = []
results = []
names = []

# definindo os parâmetros do classificador base para o ensambles
base = DecisionTreeClassifier()
num_trees = 100
max_features = 3

# criando os modelos para o VotingClassifier
bases = []
model1 = LogisticRegression(max_iter=200)
bases.append(('logistic', model1))
model2 = DecisionTreeClassifier()
bases.append(('cart', model2))
model3 = SVC()
bases.append(('svm', model3))

# Criando os elementos do pipeline

# Algoritmos que serão utilizados
reg_log = ('LR', LogisticRegression(max_iter=200))
knn = ('KNN', KNeighborsClassifier(metric="manhattan", n_neighbors=5))
cart = ('CART', DecisionTreeClassifier(criterion="gini", max_depth = 5))
naive_bayes = ('NB', GaussianNB(var_smoothing=1e-9))
svm = ('SVM', SVC(C = 1.7, kernel = "linear"))
bagging = ('Bag', BaggingClassifier(base_estimator=base, 
                                    n_estimators=num_trees))
random_forest = ('RF', RandomForestClassifier(n_estimators=num_trees, 
                                              max_features=max_features))
extra_trees = ('ET', ExtraTreesClassifier(n_estimators=num_trees, 
                                          max_features=max_features))
adaboost = ('Ada', AdaBoostClassifier(n_estimators=num_trees))
gradient_boosting = ('GB', GradientBoostingClassifier(n_estimators=num_trees))
voting = ('Voting', VotingClassifier(bases))


# Montando os pipelines

# Dataset original
pipelines.append(('LR', Pipeline([reg_log]))) 
pipelines.append(('KNN', Pipeline([knn])))
pipelines.append(('CART', Pipeline([cart])))
pipelines.append(('NB', Pipeline([naive_bayes])))
pipelines.append(('SVM', Pipeline([svm])))
pipelines.append(('Bag', Pipeline([bagging])))
pipelines.append(('RF', Pipeline([random_forest])))
pipelines.append(('ET', Pipeline([extra_trees])))
pipelines.append(('Ada', Pipeline([adaboost])))
pipelines.append(('GB', Pipeline([gradient_boosting])))
pipelines.append(('Vot', Pipeline([voting])))


# Executando os pipelines - datasets SEM transformações
for name, model in pipelines:
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %.3f (%.3f)" % (name, cv_results.mean(), cv_results.std()) 
    print(msg)

LR: 0.952 (0.008)
KNN: 0.949 (0.012)
CART: 0.947 (0.010)
NB: 0.918 (0.012)


Os resultados foram muito positivos, isso pode ser por conta de algum viés na forma de escrite de fakenews